In [1]:
from transformers import AutoConfig, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments

import torch
from torch import nn
import torch.nn.functional as F

from utils import *
from metric import compute_metrics
import wandb
import optuna
from custom_model import RBERT
from torch.utils.data import DataLoader
import os

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

helper = DataHelper(data_dir='/opt/ml/dataset/train/preprocess_train.csv')
train_idxs, val_idxs = helper.split(ratio=0.1, n_splits=5, mode='plain')[0]
train_data, train_labels = helper.from_idxs(idxs=train_idxs)
val_data, val_labels = helper.from_idxs(idxs=val_idxs)

train_data = helper.entity_tokenize(train_data, tokenizer=tokenizer)
val_data = helper.entity_tokenize(val_data, tokenizer=tokenizer)

train_dataset = RelationExtractionDataset(
    train_data, labels=train_labels)
val_dataset = RelationExtractionDataset(val_data, labels=val_labels)



In [3]:
model_config = AutoConfig.from_pretrained('klue/roberta-large',num_labels=30)
model = RBERT(model_name ='klue/roberta-large',config = model_config, dropout_rate = 0.1)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [11]:
def model_init():
    model_config = AutoConfig.from_pretrained('klue/roberta-large',num_labels=30)
    model = RBERT(model_name ='klue/roberta-large',config = model_config, dropout_rate = 0.1)
    return model

In [12]:
training_args = TrainingArguments(
    output_dir='hp_search',
    evaluation_strategy='steps',
    save_total_limit=1,
    num_train_epochs=3,
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=100,
    gradient_accumulation_steps=32,
)
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    model_init=model_init,
    compute_metrics=compute_metrics,
)


loading configuration file https://huggingface.co/klue/roberta-large/resolve/main/config.json from cache at /opt/ml/.cache/huggingface/transformers/571e05a2160c18c93365862223c4dae92bbd1b41464a4bd5f372ad703dba6097.ae5b7f8d8a28a3ff0b1560b4d08c6c3bd80f627288eee2024e02959dd60380d0
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "

In [13]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "seed": trial.suggest_int("seed", 1, 42),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 4),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 24, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 0, 0.3),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [2, 4, 8, 16, 32])
    }

In [14]:
trainer.hyperparameter_search(
    direction="maximize",
    hp_space=my_hp_space,
    # pruner=optuna.pruners.MedianPruner(
    #     n_startup_trials=2, n_warmup_steps=5, interval_steps=3
    # ),
)

[I 2021-10-03 17:35:29,389] A new study created in memory with name: no-name-79306959-892c-4633-a7d0-9cd7003bbbf6
Trial:
loading configuration file https://huggingface.co/klue/roberta-large/resolve/main/config.json from cache at /opt/ml/.cache/huggingface/transformers/571e05a2160c18c93365862223c4dae92bbd1b41464a4bd5f372ad703dba6097.ae5b7f8d8a28a3ff0b1560b4d08c6c3bd80f627288eee2024e02959dd60380d0
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer

model2 = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'class

In [ ]:
model2(tokenizer('hello'))

AttributeError: 